# Update synthetic data

- Zella King
- 2022-09-30

Sythentic data was created by taking random draws from samples for time-to-dicharge and age. 
Update synthetic data so that older people stay longer, on averaage. 

In [4]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import poisson, lognorm
import sqlalchemy as sa

In [ ]:
%matplotlib inline

## Read dummy data frame

In [69]:
#os.chdir('/Users/zellaking/Repos/HyStakes/hystakes/icu_discharges')

sqlite_engine = sa.create_engine('sqlite:///../../data/dummy.db')
df = pd.read_sql_query("SELECT id, hours_to_discharge, department, age, pulse from discharges", sqlite_engine)

# Let N be the number of patients observed
N = df.shape[0]

# Let X be the number of patients who were discharged in 24 hours
X = df[df.hours_to_discharge <= 24].shape[0]
print("Number of people discharged in 24 hours: " + str(X))
print("Total number of people observed: " + str(N))

Number of people discharged in 24 hours: 52
Total number of people observed: 444


For everyone over the mean age, we'll multiply their hours to discharge by a factor randomly selected from a normal distribution 

In [70]:
def update_los_(hours_to_discharge):
    hours_to_discharge = hours_to_discharge * np.random.normal(2, .5) 
    return hours_to_discharge


df['hours_to_discharge'] = df.apply(lambda x: update_los_(x.hours_to_discharge) if x.age > df.age.mean() else x.hours_to_discharge, axis = 1)


In [71]:
X = df[df.hours_to_discharge <= 24].shape[0]
print("Number of people discharged in 24 hours: " + str(X))
print("Total number of people observed: " + str(N))

Number of people discharged in 24 hours: 32
Total number of people observed: 444


Write dummy data out

In [72]:
df.to_sql('discharges_age_adjusted', con=sqlite_engine, if_exists='replace') 

444